In [1]:
import os
import random
import time
import argparse

from transformers import AutoTokenizer
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.optim import AdamW

from ASTE_dataloader_0503 import ASTE_Dataset, ASTE_collate_fn, load_vocab
from scheme.span_tagging import form_label_id_map, form_sentiment_id_map
from evaluate_0503 import evaluate_model, print_evaluate_dict

from utils import create_logger, ensure_dir, set_seed

from models.model_0509 import base_model
import wandb
import torch
from collections import Counter


def totally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params


def form_weight_n(n):
    if n > 6:
        weight = torch.ones(n)
        index_range = torch.tensor(range(n))
        weight = weight + ((index_range & 3) > 0)
    else:
        weight = torch.tensor([1.0, 2.0, 2.0, 2.0, 1.0, 1.0])

    return weight





ModuleNotFoundError: No module named 'ASTE_dataloader_0503'

In [ ]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--device', type=str, default='cuda', help='cuda,cpu')
    parser.add_argument('--dataset_dir', type=str, default='./data/ASTE-Data-V2-EMNLP2020')
    parser.add_argument('--saved_dir', type=str, default='saved_models')
    parser.add_argument('--saved_file', type=str, default=None)
    parser.add_argument('--pretrained_model', type=str, default='bert-base-uncased')
    
    parser.add_argument('--dataset', type=str, default='14res')
    parser.add_argument('--model_name', type=str, default='TransformerModel')

    parser.add_argument('--seed', type=int, default=42)
    # parser.add_argument('--seed', type=int, help='Random seed for reproducibility')

    parser.add_argument('--hidden_dim', type=int, default=128)
    parser.add_argument('--num_epoch', type=int, default=80,help="80,100,120")
    parser.add_argument('--batch_size', type=int, default=16)

    parser.add_argument('--lr', type=float, default=1e-5)
    parser.add_argument('--dropout_rate', type=float, default=0.5)
    parser.add_argument('--adam_epsilon', default=1e-8, type=float, help="Epsilon for Adam optimizer.")

    # loss
    parser.add_argument('--with_weight', default=True, action='store_true')
    parser.add_argument('--span_average', default=False, action='store_true')


    #
    args = parser.parse_args()
    device = torch.device(args.device if torch.cuda.is_available() else "cpu")

    destination_folder = os.path.join("saved_files_results", args.dataset_dir, args.dataset, args.version)
    args.destination_folder = destination_folder
    logger = create_logger(args, destination_folder)
    logger.info('-' * 50)
    logger.info(__file__)


    if args.seed is None:
        args.seed = random.randint(0, 2 ** 10 - 1)
    set_seed(args.seed)


    # 将所有参数及其实际使用的值记录到logger中
    for arg in vars(args):
        logger.info(f"Argument {arg}: {getattr(args, arg)}")
    logger.info('-' * 50)

    #############################################################################################
    # train_and_evaluate(base_model, args)

    tokenizer = AutoTokenizer.from_pretrained(args.pretrained_model)
 

    dataset_dir = args.dataset_dir + '/' + args.dataset
    saved_dir = args.saved_dir + '/' + args.dataset
    ensure_dir(saved_dir)

    # 加载对应词汇表
    vocab = load_vocab(dataset_dir=dataset_dir)
    # 标签映射id与标签，维度转换函数
    label2id, id2label = form_label_id_map(args.version)
    senti2id, id2senti = form_sentiment_id_map()
    # 将标签和情感的映射信息添加到词汇表中
    vocab['label_vocab'] = dict(label2id=label2id, id2label=id2label)
    vocab['senti_vocab'] = dict(senti2id=senti2id, id2senti=id2senti)


    logger.info("Load dataset...")

    train_dataset = ASTE_Dataset(file_name=os.path.join(dataset_dir, 'train_triplets.txt'),
                                 version=args.version,
                                 vocab=vocab,
                                 tokenizer=tokenizer)
    valid_dataset = ASTE_Dataset(file_name=os.path.join(dataset_dir, 'dev_triplets.txt'),
                                 version=args.version,
                                 vocab=vocab,
                                 tokenizer=tokenizer)
    test_dataset = ASTE_Dataset(file_name=os.path.join(dataset_dir, 'test_triplets.txt'),
                                version=args.version,
                                vocab=vocab,
                                tokenizer=tokenizer)

    
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=ASTE_collate_fn, shuffle=False)
    valid_dataloader = DataLoader(valid_dataset, batch_size=args.batch_size, collate_fn=ASTE_collate_fn, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, collate_fn=ASTE_collate_fn, shuffle=False)


###############################################

    # 参数设置：class_n--类别数量，weight--权重向量，
    class_n = len(label2id)
    args.class_n = class_n
    weight = form_weight_n(class_n).to(args.device) if args.with_weight else None

    logger.info("label2id:{}".format(label2id))
    logger.info("weight:{}".format(weight))

    logger.info("Load models...")

    model = base_model(model_name=args.model_name,
                       pretrained_model_path=args.pretrained_model,
                       hidden_dim=args.hidden_dim,
                       dropout=args.dropout_rate,
                       class_n=class_n,
                       span_average=args.span_average
                       ).to(args.device)

    logger.info("# parameters:{}".format(totally_parameters(model)))

    
    # optimizer = AdamW(model.parameters(),lr=args.lr,weight_decay=1e-4)
    optimizer = AdamW(model.parameters(), lr=args.lr)

    best_f1 = 0
    stop_count = 0
    increase_count = 0
    last_loss = float('inf')  # 初始化

    logger.info("Training...")
    
    for epoch in range(1, args.num_epoch + 1):

        train_loss = 0.
        total_step = 0

        epoch_begin = time.time()

        # 模型训练：设置训练模式；清除优化器的梯度；将批次数据移动到指定设备；前向传播计算损失；累积训练损失和步骤；反向传播计算梯度；优化器更新模型参数。
        for batch in train_dataloader:
            model.train()
            optimizer.zero_grad()

            inputs = {k: v.to(args.device) for k, v in batch.items()}
            outputs = model(inputs, weight)
            
            loss = outputs['loss']
            total_step += 1
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        # 评估模型
        valid_loss, valid_results = evaluate_model(model, valid_dataset, valid_dataloader,
                                                   id2senti=id2senti,
                                                   device=args.device,
                                                   version=args.version,
                                                   weight=weight)

        triplet_f1 = 100.0 * valid_results[0]['triplet']['f1']
        logger.info('Epoch:{}/{} \ttrain_loss:{:.4f}\tvalid_loss:{:.4f}\ttriplet_f1:{:.4f}% [{:.4f}s]'
                    .format(epoch, args.num_epoch, train_loss / total_step, valid_loss,
                           triplet_f1 , time.time() - epoch_begin))

        
#######################################################################################################
    logger.info('> Testing...')
    # models performance on the test set
    _, test_results = evaluate_model(model, test_dataset, test_dataloader,
                                     id2senti=id2senti,
                                     device=args.device,
                                     version=args.version,
                                     weight=weight)


    logger.info('-' * 50)
    logger.info('Dataset:{}, test_f1:{:.2f}% | version:{} lr:{} seed:{} dropout:{}'
                .format(args.dataset, test_results[0]['triplet']['f1'] * 100, args.version, args.lr,
                        args.seed, args.dropout_rate))
    evaluate_result = print_evaluate_dict(test_results)
    logger.info("evaluate result：\n{}".format(evaluate_result))


